In [16]:
from uProbeTools import *
# DICOM
import dicom, dicom.UID
from dicom.dataset import Dataset, FileDataset
import numpy as np
import datetime, time
# Constants
RAWDATA_LINE_IN_FRAME = 128
RAWDATA_SAMPLE_IN_LINE = 512
# Dataset for this exercise
DataSetName = "20170404-potatoes"
FrameRate = 12

## Let's load the images and video

In [15]:
Patates = CreateUSData(DataSetName,RAWDATA_LINE_IN_FRAME,RAWDATA_SAMPLE_IN_LINE+(525-512))
print((Patates[1][0].shape))

(128, 525)


In [37]:
def write_dicom(pixel_array,filename):
    """
    INPUTS:
    pixel_array: 2D numpy ndarray.  If pixel_array is larger than 2D, errors.
    filename: string name for the output file.
    """

    ## This code block was taken from the output of a MATLAB secondary
    ## capture.
    file_meta = Dataset()
    file_meta.MediaStorageSOPClassUID = 'Ultrasound Image Storage'
    file_meta.MediaStorageSOPInstanceUID = '1.2.840.10008.5.1.4.1.1.6.1' # Ultrasound
    file_meta.ImplementationClassUID = '1.3.6.1.4.1.9590.100.1.0.100.4.0'
    ds = FileDataset(filename, {},file_meta = file_meta,preamble="\0"*128)
    ds.Modality = 'WSD' # Workstation
    ds.ContentDate = str(datetime.date.today()).replace('-','')
    ds.ContentTime = str(time.time()) #milliseconds since the epoch
    #ds.StudyInstanceUID =  '1.3.6.1.4.1.9590.100.1.1.124313977412360175234271287472804872093'
    #ds.SeriesInstanceUID = '1.3.6.1.4.1.9590.100.1.1.369231118011061003403421859172643143649'
    #ds.SOPInstanceUID =    '1.3.6.1.4.1.9590.100.1.1.111165684411017669021768385720736873780'
    ds.SOPClassUID = 'Ultrasound Image Storage'
    ds.SecondaryCaptureDeviceManufctur = 'Kina - Python 2 - http://github.com/kelu124/kina/'
    ## These are the necessary imaging components of the FileDataset object.
    ds.SamplesPerPixel = 1
    ds.PhotometricInterpretation = "MONOCHROME2"
    ds.PixelRepresentation = 0
    ds.HighBit = 15
    ds.BitsStored = 16
    ds.BitsAllocated = 16
    ds.SmallestImagePixelValue = '\\x00\\x00'
    ds.LargestImagePixelValue = '\\xff\\xff'
    # Creating the image itself
    pixel_array = np.array(np.transpose(pixel_array))
    ds.Columns = pixel_array.shape[1]
    ds.Rows = pixel_array.shape[0]
    if pixel_array.dtype != np.uint16:
        pixel_array = pixel_array.astype(np.uint16)
    ds.PixelData = pixel_array.tostring()
    ds.save_as(filename)
    return filename

## Testing the code

In [39]:
write_dicom(Patates[1][0],"images/"+DataSetName+".dcm")

'images/20170404-potatoes.dcm'